1.Adım

Gerekli kütüphanelerin yüklenmesi

In [ ]:
!pip install pyspark

In [ ]:
!pip install pyspark[sql]

2.Adım
Bu kod bloğu, bir Spark oturumunu başlatır.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RegressionExample").getOrCreate()

3.Adım

Bu kod bloğu, veri kümesini CSV dosyasından yükler ve bir Spark DataFrame'ine dönüştürür.

In [ ]:
df= spark.read.csv('/content/house/housing.csv', header=True, inferSchema=True)

4.Adım

Bu kod bloğu, DataFrame'in ilk 20 satırını görüntüler.

In [ ]:
df.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

5.Adım

Bu kod bloğu, DataFrame'in özet istatistiklerini görüntüler. DataFrame'deki her sayısal sütun için sayı, ortalama, standart sapma, minimum ve maksimum gibi özet istatistiklerini hesaplar.

In [ ]:
df.describe().show()

+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|summary|          longitude|         latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|       households|     median_income|median_house_value|ocean_proximity|
+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|  count|              20640|            20640|             20640|             20640|             20433|             20640|            20640|             20640|             20640|          20640|
|   mean|-119.56970445736148| 35.6318614341087|28.639486434108527|2635.7630813953488| 537.8705525375618|1425.4767441860465|499.5396802325581|3.8706710029070246|206855.81690891474|           NULL|
| stddev|  2.0035317

6.Adım

Bu kod bloğu, DataFrame'deki sütunların veri tiplerini görüntüler.

In [ ]:
df.dtypes

[('longitude', 'double'),
 ('latitude', 'double'),
 ('housing_median_age', 'double'),
 ('total_rooms', 'double'),
 ('total_bedrooms', 'double'),
 ('population', 'double'),
 ('households', 'double'),
 ('median_income', 'double'),
 ('median_house_value', 'double'),
 ('ocean_proximity', 'string')]

7.Adım

Bu kod bloğu, DataFrame'deki satır sayısını görüntüler.

In [ ]:
df.count()

20640

8.Adım

Bu kod bloğu, DataFrame'in şemasını görüntüler.Şema, DataFrame'deki her sütunun adını, veri tipini ve boş değerlere izin verilip verilmediğini içerir.

In [ ]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



9.Adım

Bu kod bloğu, DataFrame'deki her sütundaki eksik değerlerin sayısını hesaplar ve görüntüler.

In [ ]:
from pyspark.sql.functions import col, sum
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



10.Adım

Bu kod bloğu, DataFrame'den eksik değerleri içeren satırları kaldırır.

In [ ]:
df = df.dropna()

11.Adım

Bu kod bloğu, StringIndexer kullanarak "ocean_proximity" sütununu sayısal bir sütuna dönüştürür. StringIndexer, kategorik bir özelliği sayısal bir dizine dönüştürmek için kullanılır.

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_proximity_index")
df = indexer.fit(df).transform(df)

12.Adım

Bu kod bloğu, OneHotEncoder kullanarak sayısal "ocean_proximity_index" sütununu one-hot encoded bir sütuna dönüştürür. OneHotEncoder, sayısal bir özelliği one-hot encoded bir vektöre dönüştürmek için kullanılır.

In [ ]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols=["ocean_proximity_index"], outputCols=["ocean_proximity_encoded"])
df = encoder.fit(df).transform(df)

13.Adım

Bu kod bloğu, VectorAssembler kullanarak özellik sütunlarını tek bir "features" sütununda birleştirir.

In [ ]:
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols=["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income", "ocean_proximity_encoded"], outputCol="features")
df= feature_assembler.transform(df)
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+---------------------+-----------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_index|ocean_proximity_encoded|            features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+---------------------+-----------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|                  3.0|          (4,[3],[1.0])|[-122.23,37.88,41...|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|                  3.0|          (4,[3],[1.0

14.Adım

Bu kod bloğu, veri kümesini eğitim ve test kümelerine ayırır.[0.7, 0.3] argümanı, eğitim ve test kümelerinin boyutlarını belirtir. seed argümanı, rastgele bölme için bir başlangıç değeri belirtir.

In [ ]:
train_data, test_data = df.randomSplit([0.7, 0.3], seed=42)

15.Adım

Bu kod bloğu, farklı regresyon modellerini içe aktarır, başlatır, eğitir, değerlendirir ve sonuçlarını görüntüler. LinearRegression, DecisionTreeRegressor, RandomForestRegressor ve GBTRegressor gibi farklı regresyon modelleri içe aktarılır. RegressionEvaluator, regresyon modellerini değerlendirmek için kullanılır. models sözlüğü, farklı regresyon modellerini ve yapılandırmalarını depolar. evaluator, regresyon modellerini değerlendirmek için bir nesne oluşturur. results sözlüğü, her model için sonuçları depolar. Kod, her model için bir döngü çalıştırır, modeli eğitir, tahminleri oluşturur, RMSE ve R-kare gibi metrikleri değerlendirir ve sonuçları results sözlüğüne kaydeder. Son olarak, kod, her model için sonuçları konsola yazdırır.

In [ ]:
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
models = {
    "Linear Regression": LinearRegression(labelCol="median_house_value", featuresCol="features"),
    "Lasso": LinearRegression(labelCol="median_house_value", featuresCol="features", elasticNetParam=1.0),
    "Ridge": LinearRegression(labelCol="median_house_value", featuresCol="features", elasticNetParam=0.0),
    "Decision Tree": DecisionTreeRegressor(labelCol="median_house_value", featuresCol="features"),
    "Random Forest": RandomForestRegressor(labelCol="median_house_value", featuresCol="features"),
    "Gradient Boosted Tree": GBTRegressor(labelCol="median_house_value", featuresCol="features")
}

evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction")

results = {}

for model_name, model in models.items():

    trained_model = model.fit(train_data)


    predictions = trained_model.transform(test_data)


    rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
    r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})


    results[model_name] = {"predictions": predictions, "rmse": rmse, "r2": r2}


for model_name, metrics in results.items():
    print(f"{model_name}:")
    print(f"  RMSE: {metrics['rmse']}")
    print(f"  R-squared: {metrics['r2']}")
    metrics['predictions'].select("prediction", "median_house_value").show(5)  # Show some predictions

Linear Regression:
  RMSE: 68906.85950355804
  R-squared: 0.6438798629719777
+------------------+------------------+
|        prediction|median_house_value|
+------------------+------------------+
|152160.49901797506|          103600.0|
|217511.24740075343|          106700.0|
|127651.04563219263|           73200.0|
|124962.06291825022|           78300.0|
|198977.66566645517|           90100.0|
+------------------+------------------+
only showing top 5 rows

Lasso:
  RMSE: 68906.85950355804
  R-squared: 0.6438798629719777
+------------------+------------------+
|        prediction|median_house_value|
+------------------+------------------+
|152160.49901797506|          103600.0|
|217511.24740075343|          106700.0|
|127651.04563219263|           73200.0|
|124962.06291825022|           78300.0|
|198977.66566645517|           90100.0|
+------------------+------------------+
only showing top 5 rows

Ridge:
  RMSE: 68906.85950355804
  R-squared: 0.6438798629719777
+------------------+---



Linear Regression	:	Doğrusal Regresyon modeli, makul bir performans gösterdi ve verilerin yaklaşık %64'ünü açıkladı. RMSE değeri, tahmin edilen ev değerleri ile gerçek ev değerleri arasındaki ortalama farkın yaklaşık 68.855 dolar olduğunu gösteriyor.

Lasso:Lasso Regresyonu modeli, Doğrusal Regresyon modeline benzer sonuçlar verdi. Bu, modelde önemli bir aşırı uyum olmadığını gösteriyor.

Ridge:	Ridge Regresyonu modeli de Doğrusal Regresyon ve Lasso modellerine benzer sonuçlar verdi. Bu, modelde çoklu doğrusallık sorununun önemli bir etkisinin olmadığını gösteriyor.

Decision Tree	:Karar Ağacı modeli, Doğrusal Regresyon modellerine göre biraz daha düşük bir performans gösterdi. Bu, modelin verileri aşırı uymuş olabileceğini gösteriyor.

Random Forest	:Rastgele Orman modeli, en iyi performans gösteren model oldu ve verilerin yaklaşık %81'ini açıkladı. RMSE değeri, tahmin edilen ev değerleri ile gerçek ev değerleri arasındaki ortalama farkın yaklaşık 49.873 dolar olduğunu gösteriyor.

Gradient Boosted Tree	: Gradyan Arttırılmış Ağaç modeli de iyi bir performans gösterdi ve verilerin yaklaşık %80'ini açıkladı. RMSE değeri, tahmin edilen ev değerleri ile gerçek ev değerleri arasındaki ortalama farkın yaklaşık 51.083 dolar olduğunu gösteriyor.

Rastgele Orman ve Gradyan Arttırılmış Ağaç modelleri, en düşük RMSE ve en yüksek R-kare değerlerine sahip olan en iyi performans gösteren modellerdi. Bu, bu modellerin bu veri kümesi için en uygun modeller olduğunu gösteriyor.
Doğrusal Regresyon, Lasso ve Ridge modelleri makul bir performans gösterdi, ancak Rastgele Orman ve Gradyan Arttırılmış Ağaç modelleri kadar iyi değildi.
Karar Ağacı modeli, aşırı uyum nedeniyle diğer modellere göre daha düşük bir performans gösterdi.